> Modelos Extensos de Lenguaje y Generación Aumentada por Recuperación

Los [modelos extensos de lenguaje](https://es.wikipedia.org/wiki/Modelo_extenso_de_lenguaje) (LLMs, del inglés *Large Language Models*), que dan vida a herramientas como *ChatGPT*, son buenos para *generar* texto, pero no siempre disponen de la información más reciente o a datos específicos que nos interesan. Aquí es donde entra en juego la [generación aumentada por recuperación](https://es.wikipedia.org/wiki/Generaci%C3%B3n_aumentada_por_recuperaci%C3%B3n) (RAG, del inglés, *Retrieval-augmented generation*): en lugar de depender solo de lo que el modelo "recuerda", podemos conectarlo a una fuente externa de conocimiento (como documentos, notas o una base de datos). Con RAG, el modelo primero recupera los fragmentos de información más relevantes, y luego los usa para generar una respuesta precisa y útil. Este enfoque hace que los LLMs sean más fiables, estén más actualizados y sean más útiles para aplicaciones del mundo real como chatbots, asistentes y sistemas de búsqueda.

Para constuir de forma sencilla un RAG, usaremos [LangChain](https://github.com/langchain-ai/langchain), que es un framework de código abierto para implementar rápidamente aplicaciones y agentes basados en LLMs usando componentes e integraciones ya preparados.

# Preparación del entorno

En principio, podrías ejecutar el *notebook* en Colab o localmente. ¿El *notebook* se está ejecutando en *Colab*?

In [ ]:
try:
    import google.colab
    running_in_colab = True
except ImportError:
    running_in_colab = False

running_in_colab

Si estás en *Colab*, es conveniente cambiar al entorno GPU...o ponerte cómodo mientras la CPU hace los cálculos.

Si no estás ejecutando en *Colab*, puede que quieras elegir una GPU si hay varias disponibles. Ignora esto si estás ejecutando en *Colab*

In [ ]:
if not running_in_colab:

    import os
    os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

## Ollama

Vamos a usar un LLM disponible gratuitamente (entrenado por otra persona). Para *ejecutarlo*, usaremos [Ollama](https://ollama.com/), que es un software para

> poner en marcha y ejecutar modelos de lenguaje grandes

Funciona con un modelo cliente-servidor, así que debemos arrancar el servidor antes de enviar cualquier petición.

- Si estás en *Colab*, se instalarán los paquetes necesarios. Puede que veas algunos <font color='red'>ERROR/WARNING</font>s. Ignóralos. y debería funcionar sin problemas.

- Si **no** estás en *Colab* deberías instalar *ollama* por tu cuenta (junto con todos los paquetes de *python* necesarios).

In [ ]:
import shutil
import pathlib
import subprocess
import time

if running_in_colab:
    !pip uninstall -y langchain 2>/dev/null || true # to avoid conflicts with new langchain version
    !pip -q install -U langchain-core langchain-community langchain-text-splitters langchain-chroma langchain-huggingface langchain-ollama llama-index-core chromadb unstructured
    !curl -fsSL https://ollama.com/install.sh | sh
    !nohup ollama serve > /dev/null 2>&1 &

else:

    log_file = pathlib.Path('ollama.log').open('w')
    proc = subprocess.Popen(
        [str(shutil.which('ollama') or pathlib.Path.home() / 'ollama' / 'bin' / 'ollama'), 'serve'],
        stdout=log_file,
        stderr=subprocess.STDOUT,
        start_new_session=True,
    )
    
# give it some time to start
time.sleep(10)

El servidor LLM (*ollama*) debería estar ya en marcha y funcionando.

## Bibliotecas de Python

El resto de `import`s necesarios se "centralizan" aquí.

In [ ]:
import requests

import numpy as np

# embedding
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

import ollama

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# database
from langchain_chroma import Chroma

# model
from langchain_ollama import OllamaLLM

# Datos

Vamos a descargar algunos datos. Aquí usamos el relato corto [Bartleby, the Scrivener](https://en.wikipedia.org/wiki/Bartleby,_the_Scrivener) de Herman Melville (disponible libremente en [Project Gutenberg](https://www.gutenberg.org/)), pero puedes usar cualquier documento de *texto plano* que prefieras (simplemente asegúrate de colocarlo en el directorio correcto o de modificar el código según haga falta).

In [ ]:
bartleby_url = 'https://www.gutenberg.org/ebooks/11231.txt.utf-8'
response = requests.get(bartleby_url)
response.raise_for_status()

data_dir = pathlib.Path('data')
data_dir.mkdir(exist_ok=True)

book_file = data_dir / 'bartleby.txt'

with book_file.open('w', encoding='utf-8') as f:
    f.write(response.text)

print(f'Downloaded book to {book_file}')

Vamos a cargar cada documento en el directorio `data` dado como un `Document` de *LangChain* (devuelve una `list` con todos los documentos).

In [ ]:
docs = DirectoryLoader('data').load()
type(docs)

<font color='red'>TO-DO</font>: ¿Cuántos documentos tienes? Revisa el contenido de uno de ellos. ¿Cuál es su tipo en Python? Intenta acceder al texto interno (el atributo importante es `page_content`).

Cada `Document` de arriba tiene que convertirse en un vector de números reales de **tamaño fijo**. Antes conseguíamos esto con *Bag-of-words*, pero podemos utilizar distintos modelos (ya entrenados) para esta tarea y aquí usaremos un *sentence transformer* (los detalles se verán en otros cursos). Este modelo es mucho más sofisticado (preserva información semántica y de contexto) que *Bag-of-words*. Los vectores resultantes de números reales de **tamaño fijo** suelen llamarse [embeddings](https://en.wikipedia.org/wiki/Embedding_(machine_learning)).

In [ ]:
embed_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
embed_model

<font color='red'>TO-DO</font>: Haz el embedding de cualquier frase que quieras (simplemente llama al `embed_model` de arriba con cualquier texto) y el de otra muy parecida. Échale un vistazo a sus vectores correspondientes y compáralos de alguna manera.

<font color='red'>TO-DO</font>: Prueba (unos cuantos) [modelos diferentes](https://huggingface.co/sentence-transformers/models) para hacer el embedding de los `Document`s. ¿Ves alguna diferencia (velocidad, rendimiento...)?


El número de *tokens* que se pueden pasar como entrada a un *sentence transformer* es limitado (eso no ocurría con *Bag-of-words*). La documentación del modelo [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) usado arriba dice

> un texto de entrada de más de 256 trozos se trunca.

Para lidiar con esto, los documentos se dividen en secuencias de (aquí) 500 *caracteres* que se **solapan** (un *token* contiene varios). Esta división de los documentos también es buena para tener una *granularidad* más fina cuando buscamos información en el documento, ya que el RAG puede localizar y devolver el segmento específico donde aparece la información relevante en lugar de un documento largo entero.

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = splitter.split_documents(docs)

<font color='red'>TO-DO</font>: Después de la división, ¿cuántos documentos tienes? ¿Qué longitudes tienen los tres primeros?

# Almacenar los documentos

Vamos a utilizar una base de datos [Chroma](https://www.trychroma.com/) para almacenar los *documentos embebidos*. Al instanciar el objeto correspondiente, necesitamos pasarle el modelo de embedding.

In [ ]:
vectordb = Chroma.from_documents(
    documents, embedding=embed_model
)

Fíjate en que el método [`from_documents`](https://reference.langchain.com/python/integrations/langchain_chroma/#langchain_chroma.Chroma.from_documents) de arriba hace el embedding y almacena los documentos de una sola vez.

<font color='red'>TO-DO</font>: ¿Qué tipo de base de datos es [Chroma](https://www.trychroma.com/)? ¿En qué se diferencia de las bases de datos tradicionales?

Puedes explotar la base de datos [Chroma](https://www.trychroma.com/) para buscar frases *similares* a una dada. Cada búsqueda en la base de datos puede (en principio, lo hará) devolver varios resultados (tantos como se soliciten a través del parámetro `k`).

In [ ]:
hits = vectordb.similarity_search(
    "I would prefer not to do it", k=3
)
len(hits)

In [ ]:
for h in hits:
    print(h.page_content[:120], '\n---')

<font color='red'>TO-DO</font>: Busca:

- Una frase que esté cerca de algo que sabes que está ahí (en el texto).

- Una frase que (probablemente) no esté.

# LLM

## Modelo

Vamos a elegir un modelo. Puedes ver [aquí](https://ollama.com/library?sort=popular) cuáles están disponibles libremente. Empecemos con

In [ ]:
# model_name = 'llama3:8b'
# model_name = 'deepseek-r1:8b'
model_name = 'llama3.2:1b' # faster download

AVISO: algunos modelos también mostrarán el proceso de pensamiento (mediante etiquetas `<think>`), lo que puede no ser muy conveniente para ciertas tareas y es posible que queramos filtrarlo.

Solo hay que descargar el modelo una vez (...por sesión si utilizas *Colab*)

In [ ]:
ollama.pull(model_name)

Una vez que el servidor está en marcha, se puede acceder a un modelo (previamente descargado) a través de un objeto de `langchain`.

In [ ]:
llm = OllamaLLM(model=model_name, temperature=0.0)

<font color='red'>TO-DO</font>: Pregúntale al modelo LLM lo que quieras (usando el método `invoke`), e.g., "¿Qué es "backpropagation"? o "¿Qué es Rick y Morty?"

<font color='red'>TO-DO</font>: Pregúntale al modelo LLM sobre el personaje principal de la novela, Bartleby. Esto lo hacemos **antes** de *aumentar* lo que el LLM sabe.

## Plantillas

Si piensas interactuar con un LLM usando siempre un determinado estilo de *prompting*, puedes crear una plantilla para ello.

In [ ]:
chat_prompt = ChatPromptTemplate([
    ('system', 'Try to rhyme every answer you give.'),
    ('human', '{user_input}'),
])

Luego se construye una cadena o *chain* (se utiliza como arriba) que *envía* (`|` puede interpretarse como $\to$) la plantilla al LLM y pasa el resultado a un *parser* que devuelve un `str`.

In [ ]:
chain = chat_prompt | llm | StrOutputParser()
print(chain.invoke('Explain backpropagation in one paragraph.'))

<font color='red'>TO-DO</font>: Experimenta con las instrucciones `system` de arriba.

<font color='red'>TO-DO</font>: Crea una nueva *chain* (guardada en una variable nueva) que se quede solo con la última palabra de cada frase y descarte el resto. Para simplificar (no necesitamos instrucciones `system`), puedes usar `PromptTemplate.from_template`.

<font color='red'>TO-DO</font>: Crea otra *chain* más encadenando (`|`) las dos anteriores para obtener una *chain* que devuelva las palabras que riman en la salida de arriba.

# "Recuperador" (*retriever*) + LLM

`langchain` proporciona una forma de construir un RAG completo de principio a fin. Solo necesitamos:

- un LLM,

- una base de datos de documentos (que se organiza usando `as_retriever` para que `langchain` sepa cómo "hablar" con ella), y

- un *prompt* que tenga en cuenta esta última.

Empecemos con el *prompt*

In [ ]:
system_prompt = (
    'Use the given context to answer the question. '
    'If you don\'t know, say so. '
    'Keep the answer under three sentences.\n\n'
    'Context:\n{context}'
)
prompt = ChatPromptTemplate.from_messages(
    [('system', system_prompt), ('human', '{input}')]
)

Una función que simplemente une los documentos.

In [ ]:
join_docs = RunnableLambda(
    lambda docs: '\n\n'.join(d.page_content for d in docs)
)

`context` e `input` son las entradas para (*rellenar*) el `prompt`, que se envía al `llm`, cuya salida se *parsea* para obtener la salida final.

In [ ]:
piped_rag_chain = (
    {                                             #  a `dict` for the prompt
        'context': vectordb.as_retriever(search_kwargs={"k": 4}) | join_docs,
        'input':   RunnablePassthrough(),
    }
    | prompt                                      # fill {context} + {input}
    | llm                                         # generate answer
    | StrOutputParser()                           # ChatMessage → str
)

<font color='red'>TO-DO</font>: Vuelve a preguntarle a esta *chain* (usando el método `invoke`) las mismas preguntas que hiciste arriba (Bartleby y lo que preguntaste antes). Compara las respuestas (ahora, **después** de *aumentar* el LLM). 

In [ ]:
if not running_in_colab:
    proc.terminate()   # or proc.kill() for a hard stop
    proc.wait()
# log.close()

<font color='red'>TO-DO</font> : Prueba un modelo de lenguaje distinto (hay un par de ellos comentados en el código donde se define `model_name`). Ojo con el tamaño del modelo (*7b*, *8b*...). Los modelos grandes pueden tardar **mucho** en dar una respuesta... o incluso colapsar la memoria de la GPU. El sufijo *b* en esos modelos significa *billones* (americanos). El *Toy GPT* que entrenamos antes tenía... 0.2 **m**illones.

# Preguntas de ejemplo

## ¿Cuál es el beneficio principal de añadir recuperación (retrieval) a un sistema basado en LLM?
- [ ] Elimina la necesidad de diseñar *prompts*.
- [ ] Aporta primero contexto externo relevante, haciendo las respuestas más precisas y actualizadas.
- [ ] Reescribe permanentemente los pesos del modelo con hechos nuevos.
- [ ] Obliga al modelo a responder solo con citas.

## ¿Qué representa un *embedding* para un fragmento de texto?
- [ ] Una compresión perfecta y sin pérdida del texto original.
- [ ] El conteo únicamente de palabras y signos de puntuación.
- [ ] Un vector numérico que captura el significado de modo que textos similares queden cerca entre sí.
- [ ] Una lista de palabras clave en orden alfabético.
